In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
#from scipy import stats
import sklearn
import sklearn.svm as svm
from scipy.stats import norm, skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import   BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestClassifier,  GradientBoostingClassifier
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import csv
from collections import Counter
from sklearn.linear_model import LinearRegression


D:\app\anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [15]:
data = pd.read_csv("C:/Users/wxz/Desktop/face/01-data/01-data/01class.csv")
label= data['label']
data = data.drop(['label'],axis=1)
for i in range(72):
    temp = data['f{}'.format(i)]
    temp = (temp - np.mean(temp))/np.var(temp)
    temp = (temp - min(temp))/(max(temp)-min(temp))
    data['f{}'.format(i)] = temp
data = np.array(data)
label = np.array(label)


In [16]:
data[np.isnan(data)]=0

In [19]:
np.isnan(data)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [17]:
train_data,test_data, train_label, test_label = train_test_split(data,label,test_size=0.2,random_state=2020)
train_data1 ,train_data2,train_label1,train_label2 = train_test_split(train_data,train_label,test_size=0.2,random_state=2020)

In [20]:
model_rfc = RandomForestClassifier(n_estimators=3000, criterion='gini', max_depth=None,
                                   min_samples_split=2,min_samples_leaf=1,
                                   min_weight_fraction_leaf=0.0,
                                   max_features='auto',
                                   max_leaf_nodes=None, bootstrap=True,
                                   oob_score=False, n_jobs=1, random_state=None, verbose=0,
                                   warm_start=False, class_weight=None)
model_rfc.fit(train_data1,train_label1)
pre1 = model_rfc.predict_proba(train_data2)

In [21]:
model_xgb = xgb.XGBClassifier(colsample_bytree=0.4603, gamma=0.0468,
                             learning_rate=0.05, max_depth=3,
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_xgb.fit(train_data1,train_label1)
pre2 = model_xgb.predict_proba(train_data2)

In [22]:
model_lgb = lgb.LGBMClassifier(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
model_lgb.fit(train_data1,train_label1)
pre3 = model_lgb.predict_proba(train_data2)

In [23]:
model_dtc = sklearn.tree.DecisionTreeClassifier(criterion='gini', splitter='best',max_depth=None, min_samples_split=2, min_samples_leaf=1,min_weight_fraction_leaf=0.0, max_features=None,random_state=None,max_leaf_nodes=None, min_impurity_decrease=1e-07, class_weight=None)
model_dtc.fit(train_data1,train_label1)
pre4 = model_dtc.predict_proba(train_data2)

In [24]:
model_svm = svm.SVC(C = 32,gamma = 0.5,kernel='rbf', probability = True)
model_svm.fit(train_data1,train_label1)
pre5 = model_svm.predict_proba(train_data2)

In [25]:
meta_model = LinearRegression()

In [26]:
label1 = np.argmax(pre1, axis=1)
acc1 = (label1==train_label2).sum()/len(train_label2)
acc1

0.5833333333333334

In [27]:
label2 = np.argmax(pre2, axis=1)
acc2 = (label2==train_label2).sum()/len(train_label2)
acc2

0.75

In [28]:
label3 = np.argmax(pre3, axis=1)
acc3 = (label3==train_label2).sum()/len(train_label2)
acc3

0.6666666666666666

In [29]:
label4 = np.argmax(pre4, axis=1)
acc4 = (label4==train_label2).sum()/len(train_label2)
acc4

0.75

In [30]:
label5 = np.argmax(pre5, axis=1)
acc5 = (label5==train_label2).sum()/len(train_label2)
acc5

0.6666666666666666

In [31]:
pre6 = pre1+pre2+pre3+pre4+pre5
label6 = np.argmax(pre6, axis=1)
acc6 = (label6==train_label2).sum()/len(train_label2)
acc6

0.75

In [32]:
out_put = [label1,label2,label3,label4,label5,label6]
out_put = np.array(out_put)
out_put= out_put.T
out_put

array([[0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 1],
       [1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0]], dtype=int64)

In [33]:
meta_model.fit(out_put,train_label2)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [34]:
l1 = model_rfc.predict_proba(test_data)
l2 = model_xgb.predict_proba(test_data)
l3 = model_lgb.predict_proba(test_data)
l4 = model_dtc.predict_proba(test_data)
l5 = model_svm.predict_proba(test_data)
l6 = l1+l2+l3+l4+l5
l1 = np.argmax(l1, axis=1)
l2 = np.argmax(l2, axis=1)
l3 = np.argmax(l3, axis=1)
l4 = np.argmax(l4, axis=1)
l5 = np.argmax(l5, axis=1)
l6 = np.argmax(l6, axis=1)
all_l = np.array([l1,l2,l3,l4,l5,l6])
all_l = all_l.T

In [37]:
pre_test = meta_model.predict(all_l)
acc = ((abs(pre_test-test_label))<=1).sum()/len(test_label)
acc

0.7142857142857143

In [42]:
pre_test

array([ 2.63157895e-01,  2.10526316e-01,  2.63157895e-01, -1.26315789e+00,
        2.63157895e-01, -1.26315789e+00,  2.10526316e-01,  1.00000000e+00,
        2.63157895e-01,  1.73684211e+00,  2.63157895e-01, -8.32667268e-17,
        2.47368421e+00, -1.26315789e+00])

In [43]:
test_label

array([1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1], dtype=int64)

In [44]:
label6

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int64)